# 1. Segmentation

In the final step, we will perform the segmentation and wear measurement. Your task will be to receive unseen images for the model, perform segmentation on these images, and finally measure the wear thickness to plot the corresponding VB curve for each cutting edge.

For having a better results in segmentation, you will receive a weight that has been trained on a larger dataset, resulting in improved accuracy. However, before proceeding with the segmentation task, certain preprocessing steps are required to ensure optimal results.

If you have any doubts about the previous section, please go ahead and book a consultation.

In [ ]:
from keras.models import load_model
from pathlib import Path
model_path = Path("..") / "model.h5"

model = load_model(model_path)

2025-06-13 16:18:16.492042: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


# 1.1. Image Alignment
Before delving into segmentation, it's essential to address potential issues stemming from image acquisition, such as image shifting. To mitigate these issues and ensure consistency across images, we will employ image alignment techniques.

By aligning the images based on a reference or source image, we can ensure that the cutting edge, or the region of interest, remains roughly in the same position across all images. This not only simplifies the segmentation process but also improves the accuracy of subsequent measurements, such as wear thickness calculation.

With the images properly aligned, we can proceed confidently to the segmentation phase, where we'll extract meaningful insights regarding wear thickness and plot the corresponding VB curve.

You can find the images and the weight here: https://bwsyncandshare.kit.edu/s/kpdaZs6EtcM5Bpn

The test dataset comprises a complete set of cutting-edge images, starting from the beginning to the end of the tool's lifespan (approximately 200μm). As the tool has four cutting edges, four images are captured after each data collection step. The collection step occurs after every 10 tests, meaning the milling process is performed 10 times then four images are taken. This sequence is then repeated.

***Please refresh the images folder to view the images in the notebook (Download the updated images from ilias)

<div style="text-align: center;">
    <img src="Images/image-14.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

<div style="text-align: center;">
    <img src="Images/image-15.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

* TASK: Try using the OpenCV library to implement the alignment method.

- we believe scaling is not necessary. only trans and rotation. so euclidean motion model is sufficient

In [1]:
import cv2 
import os 
import numpy as np 
from pathlib import Path
from tqdm import tqdm

# Folder with the images
int_folder = Path("..") / "test" / "Test_Dataset"

# Load image filenames
image_files = sorted([filename for filename in os.listdir(int_folder) if filename.endswith(('.jpg', '.jpeg', '.png'))])
image_files_source = [image_files[0]]
image_files_test = image_files[1:]

# Load reference image
img_path = os.path.join(int_folder, image_files_source[0])
imgRef = cv2.imread(img_path)
imgRef_gray = cv2.cvtColor(imgRef, cv2.COLOR_BGR2GRAY)
sz = imgRef.shape

# Use Euclidean motion model: rotation + translation only
warp_mode = cv2.MOTION_EUCLIDEAN

# Initialize 2x3 warp matrix to identity
warp_matrix = np.eye(2, 3, dtype=np.float32)

# ECC optimization parameters
number_of_iterations = 1000
termination_eps = 1e-6
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations, termination_eps)

# Create output folder
aligned_folder = int_folder / "aligned"
aligned_folder.mkdir(exist_ok=True)

for filename in tqdm(image_files_test): 
    img_path = os.path.join(int_folder, filename)
    imgTest = cv2.imread(img_path)
    imgTest_gray = cv2.cvtColor(imgTest, cv2.COLOR_BGR2GRAY)

    # Estimate warp matrix using ECC
    try:
        cc, warp_matrix_est = cv2.findTransformECC(
            imgRef_gray, imgTest_gray, warp_matrix.copy(), warp_mode, criteria
        )

        # Warp the image
        aligned_img = cv2.warpAffine(
            imgTest, warp_matrix_est, (sz[1], sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP
        )

        # Save aligned image
        aligned_filename = aligned_folder / f"aligned_{filename}"
        cv2.imwrite(str(aligned_filename), aligned_img)
        # print(f"Aligned: {aligned_filename}")
    
    except cv2.error as e:
        print(f"Failed to align {filename}: {e}")

  9%|▉         | 25/275 [05:38<56:26, 13.54s/it]  


KeyboardInterrupt: 

# 1.2. ROI Extraction

In our data preprocessing step, we have already identified the regions of interest (ROI) within the original images. Now, our task is to crop these ROIs from the test dataset as well. This is crucial for reducing the size of the images, which can streamline subsequent processing steps and improve computational efficiency.

By extracting only the relevant regions from the original images, we focus our computational resources on the areas that are most important for our analysis. This not only speeds up processing but also helps in maintaining the integrity of the data by minimizing irrelevant information.

* TASK: Crop the Region of Interest (ROI) area, as performed in the training step

In [ ]:
# Input and output directories
int_folder = Path("..") / "test" / "Test_Dataset" / "aligned"
out_folder = Path("..") / "test" / "Test_Dataset" / "cropped"
out_folder.mkdir(exist_ok=True)


def crop_imgs(input_dir: Path, output_dir) -> None:
   for img_path in tqdm(input_dir.glob(".jpg")):
      #crop the center section ~> 1400W * 1840H
      with Image.open(img_path) as img:
         width, height = img.size
         left = (width - 1400) // 2
         top = (height - 1840) // 2
         right = left + 1400
         bottom = top + 1840
         cropped = img.crop((left, top, right, bottom))
         cropped.save(output_dir / img_path.name)
   return

crop_imgs(int_folder, out_folder)
print("Cropping completed.")

# 1.3. Image resizing

In order to utilize the provided segmentation model effectively, you need to resize the images to a specific size, in this case, 512 x 512 pixels. Resizing images to a consistent dimension ensures compatibility with the model's input requirements and helps maintain the aspect ratio of the original images.

* TASK: Implement the code to resize the images to the desired dimensions

<div style="text-align: center;">
    <img src="Images/image-17.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

* TASK: Resize the images to match the dimensions of the training dataset

In [ ]:
from PIL import Image
# Input and output directories
int_folder = Path("..") / "test" / "Test_Dataset" / "cropped"
out_folder = Path("..") / "test" / "Test_Dataset" / "resized"
out_folder.mkdir(exist_ok=True)

def crop_imgs(input_dir: Path, output_dir) -> None:
   for img_path in tqdm(input_dir.glob(".jpg")):
      #crop the center section ~> 1400W * 1840H
      with Image.open(img_path) as img:
         width, height = img.size
         left = (width - 1400) // 2
         top = (height - 1840) // 2
         right = left + 1400
         bottom = top + 1840
         cropped = img.crop((left, top, right, bottom))
         cropped.save(output_dir / img_path.name)
   return
        

print("Resizing completed.")

# 1.4. Segmenattion Model Implemanation

Now that we have completed all the preprocessing steps, our images are prepared for segmentation. By utilizing the provided weight, you can generate the segmentation results for these preprocessed images.

ATTENTION: YOU NEED TO RESIZE THE SEGMENTATION RESULT TO THE ORIGINAL SIZE TO KEEP THE PIXEL RATIO

* TASK: Implement the segmentation code

In [ ]:
# Input and output directories
int_folder = "Input Directory"
out_folder = "Output Directory"

for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')): 
        ~~~

print("Prediction completed.")

Here is an example of the segmentation results obtained after applying the segmentation model.

<div style="text-align: center;">
    <img src="Images/image-19.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

# 2. Wear Measurement
In this step, we are going to find out the thickness of the wear area in each image, so we can see a trend of how the wear is changing by continuing the milling process. Of course, the segmentation results are not 100% perfect, but as we just need to focus on the maximum thickness in each image, we are on the safe side. At the end of this step, there will be a value for the maximum thickness for each image. The pixel ratio that we have in the image is 1.725 μm per pixel. In this section, you have complete freedom to implement your own solution, however I will provide some hints to get you started. Let's dive into the last step :)

# 2.1. label the Target Cluster

If you check the results, you will probably see some misdetections where the model labeled areas as wear, which are not. To eliminate these artifacts and focus on the target wear area, you can implement a clustering method to identify the target area. Since the images are already aligned and the cutting edge coordinates are similar, you can also define a bounding box to locate the target cluster more accurately.

<div style="text-align: center;">
    <img src="Images/image-20.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

* TASK: Try to locate the right segmentation area. For this purpose, you can use findContours from OpenCV, but you can also implement your own solution

In [ ]:
def find_contour_indices(contours):
    valid_indices = []
    for i in range(len(contours)):
        if len(contours[i]) > 50:
            for j in range(len(contours[i])):
                    ~~~~
                    break  
    return valid_indices

# 2.2. VB Measuring

In this step, we will measure the maximum thickness of the wear. You can implement a combination of functions from OpenCV, such as findContours and Canny edge detection, to find the thickness. It's important to calculate the orthogonal distance (line 1 in Max VB) each time, not the horizontal distance (line 2 in Max VB).

<div style="text-align: center;">
    <img src="Images/image-21.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

* TASK: Try to measure the maximum wear thickness 

In [ ]:
directory = "Input Directory"
mg_name = []
VBmax = []

for filename in os.listdir(directory):
    img_name.append(filename) 
    img = cv2.imread(os.path.join(directory,filename), cv2.IMREAD_GRAYSCALE)
    ret,thresh = cv2.threshold(img,127,255,0)
    contours,hierarchy = cv2.findContours(thresh, 1, 2)
    result = find_contour_indices(contours)
    print(result)
    min_max_list = []
    
    for i in result:
        ~~~

# 2.3. VB Curve

After extracting the maximum thickness of the wear in each image, as already mentioned, the images are for 4 cutting edges, each after 10 cuts. You can plot the VB curve for each cutting edge. 

<div style="text-align: center;">
    <img src="Images/image-22.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

* TASK: plot the VB curve

In [ ]:
WB = Workbook() 
Sheet1 = WB.add_sheet ('VB') 

Sheet1.write (0,0,,~~) 
Sheet1.write (1,0,,~~)
Sheet1.write (2,0,,~~) 
Sheet1.write (3,0,,~~)
Sheet1.write (4,0,~~) 


for i in range(len(VBmax_new)): 
    Sheet1.write (0,i+1,str(i+1) +'_Track',style) 
    Sheet1.write (1,i+1,float(VBmax_new[i][0]))
    Sheet1.write (2,i+1,float(VBmax_new[i][1]))
    Sheet1.write (3,i+1,float(VBmax_new[i][2]))
    Sheet1.write (4,i+1,float(VBmax_new[i][3])) 

WB.save (r"Output Directory") 

# Extracting the tooth numbers
tooth_numbers = [f'{i}th Tooth' for i in range(1, len(VBmax_new) + 1)]

# Creating x-axis labels for all 5 cuts
x_labels = [f'{i}' for i in range(1, ~~~,5)]

# Transposing VBmax_new for easy plotting
vb_values_transposed = list(map(list, zip(*VBmax_new)))

# Define marker colors
marker_colors = [~~~]

# Plotting all 5 cuts for each tooth
plt.figure(figsize=(12, 8))
for i, (vb_values, color) in enumerate(zip(vb_values_transposed, marker_colors)):
    ~~~
# Show the plot
plt.show()

# 2.4. Data Filtering

After plotting the VB curve, you might notice that some results do not follow the trend and are outliers. The most obvious ones are those that are higher than the previous and the next points, which is not possible. So, try to filter the data and eliminate the artifact results.

To detect and remove such anomalies, you can apply unsupervised learning techniques like DBSCAN (Density-Based Spatial Clustering of Applications with Noise) or use smoothing methods such as LOWESS (Locally Weighted Scatterplot Smoothing) from the statsmodels.nonparametric module.

Below is an example of filtered results that you can use for comparison with your own data. (Please re-dowload the folder 'Images' from ilias)

<div style="text-align: center;">
    <img src="Images/image-25.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

* TASK: Implement a clustering method for data filtering and eliminate the outliers

# 2.5. Video Processing

In addition to still images, you may also have videos of the cutting tool rotating slowly in front of a camera. In these videos, the tool rotates multiple times, so each cutting edge appears several times across different frames.

Your task is to automatically detect the frames corresponding to each cutting edge, starting with identifying the first clear frame for each of the four cutting edges. Then, extract all frames belonging to each respective cutting edge throughout the video. Finally, apply your existing wear characterization algorithm to detect and measure the wear area from the selected frames.

Below is a step-by-step guideline to help you structure your solution:

* Extract all frames from the video using OpenCV or similar tools.

* Identify the first frame where each of the four cutting edges is clearly visible, ideally matching the orientation and appearance of your existing image dataset.

* Track and collect all frames that show each respective cutting edge during subsequent tool rotations.

* Use techniques such as:

    * Edge detection + template matching (cv2.matchTemplate()), or

    * Image similarity metrics like SSIM (Structural Similarity Index) or histogram comparison to match frames with known reference images.

* Group the detected frames into four categories, each representing one cutting edge.

* Apply your wear detection algorithm to the grouped frames to measure and analyze wear over time or rotation.

By completing this task, you will automate the analysis of video data, enabling consistent wear monitoring of each cutting edge from continuous tool rotation footage.

Link to videos: https://bwsyncandshare.kit.edu/s/kpdaZs6EtcM5Bpn






<div style="text-align: center;">
    <img src="Images/image-30.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>